In [2]:
from neo4j import GraphDatabase
import graphdatascience

/Users/jakob/Development/Bachelorarbeit/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Time Tree Embedding

In [3]:
gds = graphdatascience.GraphDataScience("neo4j://localhost:7687", auth=("neo4j", "bachelorarbeit"))
gdsdriver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "bachelorarbeit")).session(database="timetree")

In [4]:
# Calendar 1 is January, 2 is February, etc.
calendar = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
            7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}

hours = 24

minutes = [0, 15, 30, 45]

def run_query(query, parameters=None):
    with gdsdriver as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

# Root Knoten der als Wurzel für den Baum dient
def add_root_node(tx):
    return tx.run(
        "CREATE (root:Root)"
    )


def add_has_year_relationship(tx, year):
    return tx.run(
        "MATCH (r:Root) "
        "CREATE (y:Year {value: $year}) "
        "CREATE (r)-[:HAS_YEAR]->(y)",
        year=year
    )


def add_has_month_relationship(tx, year, month):
    return tx.run(
        "MATCH (y:Year {value: $year}) "
        "CREATE (m:Month {value: $month}) "
        "CREATE (y)-[:HAS_MONTH]->(m)",
        year=year, month=month
    )


def add_has_month(tx, year, month):
    return tx.run(
        "MATCH (y:Year {value: $year}) "
        "CREATE (m:Month {value: $month}) "
        "CREATE (y)-[:HAS_MONTH]->(m)",
        year=year, month=month
    )


def add_has_day(tx, month, day):
    return tx.run(
        "MATCH (m:Month {value: $month}) "
        "CREATE (d:Day {value: $day}) "
        "CREATE (m)-[:HAS_DAY]->(d)",
        month=month, day=day
    )

def add_has_hour(tx, month, day, hour):
    return tx.run(
        "CREATE (h:Hour {value: $hour})"
        "WITH (h)"
        "MATCH (m:Month{value: $month})-[:HAS_DAY]->(d:Day{value: $day})"
        "MERGE (d)-[:HAS_HOUR]->(h)",
        month=month, day=day, hour=hour
    )


def next_relationship_in_month(tx, day, month):
    return tx.run(
        "MATCH (m:Month {value: $month})-[:HAS_DAY]->(d:Day {value: $day}) "
        "WITH d, d.value AS currentDay, d.value + 1 AS nextDay, m "
        "MATCH (m)-[:HAS_DAY]->(n:Day {value: nextDay})"
        "MERGE (d)-[:NEXT]->(n)",
        day=day, month=month
    )


def next_relationship_next_month(tx, day, month):
    return tx.run(
        """
        MATCH (m:Month {value: $month})-[:HAS_DAY]->(d:Day {value: $day})
        WITH m, d, m.value AS currentMonth, m.value + 1 AS nextMonth
        MATCH (c:Month {value: nextMonth})-[:HAS_DAY]->(k:Day {value: 1})
        MERGE (d)-[:NEXT]->(k)
        """,
        day=day, month=month
    )


def has_minute(tx, month, day, hour, minute):
    return tx.run(
        "CREATE (m:Minute {value: $minute})"
        "WITH (m)"
        "MATCH (month:Month{value: $month})-[:HAS_DAY]->(d:Day{value: $day})-[:HAS_HOUR]->(h:Hour{value: $hour})"
        "CREATE (h)-[:HAS_MINUTE]->(m)",
        hour=hour, minute=minute, month=month, day=day
    )

def create_time_tree(year, driver):
    with driver as session:
        add_root_node(session)
        add_has_year_relationship(session, year)

        for month, days in calendar.items():
            add_has_month(session, year, month)
            for day in range(1, days + 1):
                add_has_day(session, month, day)
                for hour in range(1, hours + 1):
                    add_has_hour(session, month, day, hour)
                    for minute in minutes:
                        has_minute(session, month, day, hour, minute)
            for day in range(1, days + 1):
                if day < days:
                    next_relationship_in_month(session, day, month)
        for month, days in calendar.items():
            if month < 12:
                next_relationship_next_month(session, days, month)
                


connection_timetree_trip = """CALL apoc.periodic.iterate(
  "
  MATCH (t:Trip)
  RETURN t, 
         t.validFrom.year AS year, 
         t.validFrom.month AS month, 
         t.validFrom.day AS day, 
         t.validFrom.hour AS hour, 
         CASE 
           WHEN t.validFrom.minute % 15 = 0 THEN t.validFrom.minute
           ELSE toInteger(floor(t.validFrom.minute / 15.0) * 15)
         END AS roundedMinute
  ",
  "
  MATCH (:Year {value: year})-[:HAS_MONTH]->(:Month {value: month})-[:HAS_DAY]->(:Day {value: day})-[:HAS_HOUR]->(:Hour {value: hour})-[:HAS_MINUTE]->(mi:Minute {value: roundedMinute})
  MERGE (t)-[:STARTED_AT]->(mi)
  ",
  {batchSize: 1000, parallel: false}
) YIELD batches, total
RETURN batches, total;
"""

In [5]:
create_time_tree(2017, gdsdriver)
run_query(connection_timetree_trip)

SessionError: Session closed

In [ ]:
gdsdriver.close()